In [1]:
#note: temp readings are 20s apart

import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

os.chdir('data')

In [2]:
dfs = []# list of dataframes

for file in os.listdir():
    try:
        df = pd.read_excel(file, sheet_name="DATA", usecols='C,D', skiprows = 1)
        df.loc[df.shape[0]] = [None, None]# to ensure that there are breaks between each recorded shower
        dfs.append(df)
    except:
        print(file)

data = pd.concat(dfs, ignore_index = True)# concatenating all data into one dataframe
data.columns = ['temp', 'label']

.DS_Store


In [3]:
start_pts = []#list where the first item is 1 for start and 0 for not start
end_pts = []#same list but for ends
for i, row in data.iterrows():
    if not pd.isnull(row['label']):
        if row['label'].startswith('Start'):
            start_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
        if row['label'].startswith('End'):
            end_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
            
initial_length = len(start_pts)
if initial_length != len(end_pts):
    print("inconsistent labels!")

random_start_indices = random.sample(range(len(data) - 9), k =  len(data) - 9)
random_end_indices = random.sample(range(len(data) - 9), k = len(data) - 9)

for i in random_start_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('Start'))):
        start_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(start_pts) == initial_length * 2:
            break

for i in random_end_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('End'))):
        end_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(end_pts) == initial_length * 2:
            break

In [6]:
start_data = tf.convert_to_tensor(start_pts)#tensors holding the label, followed by a list of 10 temperatures
start_data = tf.random.shuffle(start_data)

end_data = tf.convert_to_tensor(end_pts)
end_data = tf.random.shuffle(end_data)

In [8]:
data_length = start_data.shape[0]

start_train = start_data[:int(data_length * .7)]
start_val = start_data[int(data_length * .7):int(data_length * .85)]
start_test = start_data[int(data_length * .85):]

end_train = end_data[:int(data_length * .7)]
end_val = end_data[int(data_length * .7):int(data_length * .85)]
end_test = end_data[int(data_length * .85):]

In [22]:
start_model = Sequential(
    [
        BatchNormalization(),
        Dense(16, activation = 'relu', input_shape=(10,)),
        Dense(16, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

start_model.compile(loss='mse', metrics=['mae'])
start_model.fit(start_train[:,1:], start_train[:,1], epochs=20)

Epoch 1/20
3/3 [==============================] - 0s 2ms/step - loss: 6051.6504 - mae: 77.6081
Epoch 2/20
3/3 [==============================] - 0s 2ms/step - loss: 6044.1099 - mae: 77.5595
Epoch 3/20
3/3 [==============================] - 0s 2ms/step - loss: 6038.4209 - mae: 77.5230
Epoch 4/20
3/3 [==============================] - 0s 2ms/step - loss: 6034.5439 - mae: 77.4981
Epoch 5/20
3/3 [==============================] - 0s 2ms/step - loss: 6030.9663 - mae: 77.4755
Epoch 6/20
3/3 [==============================] - 0s 2ms/step - loss: 6027.7788 - mae: 77.4552
Epoch 7/20
3/3 [==============================] - 0s 2ms/step - loss: 6024.7275 - mae: 77.4358
Epoch 8/20
3/3 [==============================] - 0s 2ms/step - loss: 6022.6982 - mae: 77.4225
Epoch 9/20
3/3 [==============================] - 0s 2ms/step - loss: 6019.4219 - mae: 77.4021
Epoch 10/20
3/3 [==============================] - 0s 2ms/step - loss: 6018.1128 - mae: 77.3938
Epoch 11/20
3/3 [==============================] 

In [25]:
start_val = model.predict([start_val[:,1:]])
start_val.shape

Consider rewriting this model with the Functional API.


ValueError: in user code:

    File "c:\users\bring\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "c:\users\bring\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\bring\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "c:\users\bring\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "c:\users\bring\appdata\local\programs\python\python38\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\users\brin

In [18]:
np.mean(np.absolute(y_test - y_predicted[0,:,0]))

NameError: name 'np' is not defined